# Getting points for fantasy defense through previous week
nflfastr, which was the original package that was cloned into nfl_data_py, is much more reliable in updating the data. So we need to go grab that in R Studio, then put them into CSVs that we retrieve below.<br>

This notebook will calculate fantasy points scored by every defense in the previous season and the current season so that we can derive L8 (last-eight-game) variables for our model.

In [1]:
##Defensive fantasy points for FanDuel and DraftKings
# Sacks = 1
# Opponent-fumbles recovered = 2
# Return touchdowns = 6
# Extra Point Return = 2
# Safeties = 2
# Blocked Punt/Kick = 2
# Interceptions made = 2
# 0 points allowed = 10
# 1-6 points allowed = 7
# 7-13 points allowed = 4
# 14-20 points allowed = 1
# 21-27 points allowed = 0
# 28-34 points allowed = -1
# 35+ points allowed = -4

In [2]:
import numpy as np
import pandas as pd
#import nfl_data_py as nfl
from functions import get_current_weekday, calculate_nfl_week, get_next_sunday, get_current_year

In [3]:
day = get_current_weekday()

In [4]:
date_string = get_next_sunday(day)

In [5]:
week = calculate_nfl_week(date_string)

In [6]:
season = get_current_year()

In [7]:
#Play-by-play data last three seasons
nfl_pbp = pd.read_csv('pbp_data_' + str(season) + '_' + str(week) + '.csv', low_memory=False)

In [8]:
#Weekly data last two seasons
weekly_df = pd.read_csv('weekly_data_' + str(season) + '_' + str(week) + '.csv')

This is the list of columns we extracted from the nfl play-by-play data

In [9]:
defense_cols = ['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team', 'season_type', 'week', 'posteam', 'posteam_type', 'defteam',\
'sp', 'play_type', 'field_goal_result', 'extra_point_result', 'two_point_conv_result', 'td_team', 'punt_blocked', 'interception',\
'fumble_forced', 'fumble_not_forced', 'safety', 'fumble_lost', 'own_kickoff_recovery', 'own_kickoff_recovery_td', 'sack', 'touchdown',\
                'pass_touchdown', 'rush_touchdown', 'return_touchdown', 'extra_point_attempt',\
'two_point_attempt', 'field_goal_attempt', 'kickoff_attempt', 'punt_attempt', 'fumble', 'forced_fumble_player_1_team', 'forced_fumble_player_2_team',\
'fumbled_1_team', 'fumbled_2_team', 'fumble_recovery_1_team', 'fumble_recovery_2_team', 'return_team', 'defensive_two_point_attempt',\
'defensive_two_point_conv', 'defensive_extra_point_attempt', 'defensive_extra_point_conv', 'season', 'drive_ended_with_score', 'away_score',\
'home_score', 'location', 'result', 'total', 'spread_line', 'total_line', 'div_game', 'roof', 'surface', 'pass', 'rush',\
                'posteam_score', 'posteam_score_post', 'defteam_score', 'defteam_score_post', 'desc']	

In [10]:
nfl_pbp.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NaN,NaN,NaN,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.443521,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,1.468819,NaN,NaN,NaN,NaN,NaN,0.440373,-44.037291
3,89,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.492192,0.727261,6.988125,6.0,0.60693,0.227598,0.389904,61.009598
4,115,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,1,-0.325931,NaN,NaN,NaN,NaN,NaN,0.443575,-44.357494


In [11]:
defense_df = nfl_pbp[defense_cols]

In [12]:
defense_df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,div_game,roof,surface,pass,rush,posteam_score,posteam_score_post,defteam_score,defteam_score_post,desc
0,1,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NaN,NaN,NaN,...,0,outdoors,fieldturf,0,0,NaN,NaN,NaN,NaN,GAME
1,43,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,outdoors,fieldturf,0,0,0.0,0.0,0.0,0.0,9-J.Tucker kicks 68 yards from BAL 35 to NYJ -...
2,68,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,outdoors,fieldturf,0,1,0.0,0.0,0.0,0.0,(14:56) 32-Mi.Carter left end to NYJ 41 for 19...
3,89,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,outdoors,fieldturf,1,0,0.0,0.0,0.0,0.0,"(14:29) (No Huddle, Shotgun) 19-J.Flacco pass ..."
4,115,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0,outdoors,fieldturf,0,1,0.0,0.0,0.0,0.0,(14:25) (No Huddle) 32-Mi.Carter left end to N...


In [13]:
defense_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110040 entries, 0 to 110039
Data columns (total 65 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   play_id                        110040 non-null  int64  
 1   game_id                        110040 non-null  object 
 2   old_game_id                    110040 non-null  int64  
 3   home_team                      110040 non-null  object 
 4   away_team                      110040 non-null  object 
 5   season_type                    110040 non-null  object 
 6   week                           110040 non-null  int64  
 7   posteam                        104051 non-null  object 
 8   posteam_type                   104051 non-null  object 
 9   defteam                        104051 non-null  object 
 10  sp                             110040 non-null  int64  
 11  play_type                      106814 non-null  object 
 12  field_goal_result             

In [14]:
weekly_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,1,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,10.38,10.38
1,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,2,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,9.40,9.40
2,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,3,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,14.74,14.74
3,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,4,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,25.40,25.40
4,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,TB,2022,5,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,19.74,19.74


In [15]:
weekly_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12518 entries, 0 to 12517
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    12518 non-null  object 
 1   player_name                  12518 non-null  object 
 2   player_display_name          12518 non-null  object 
 3   position                     12518 non-null  object 
 4   position_group               12518 non-null  object 
 5   headshot_url                 12456 non-null  object 
 6   recent_team                  12518 non-null  object 
 7   season                       12518 non-null  int64  
 8   week                         12518 non-null  int64  
 9   season_type                  12518 non-null  object 
 10  opponent_team                12518 non-null  object 
 11  completions                  12518 non-null  int64  
 12  attempts                     12518 non-null  int64  
 13  passing_yards   

# Functions
We're going to do a lot of grouping of dataframes, and these two functions will be useful. The first one can be used when we're aggregating multiple columns. The second one can be used when we just aggregate a single column.<br>

The third function basically expands our base dataframe every time we aggregate a new column.<br>

First, we'll create three lists that will be used as arguments to the functions, and below the functions is the aggregration dictionary that will be used as an argument the first time we use the group_cols function.

In [16]:
cols_to_group_defteam = ['season', 'week', 'game_id', 'defteam'] #defteam is the last item in the list
cols_to_group_tdteam = ['season', 'week', 'game_id', 'td_team'] #td team is the last item in the list
cols_to_group_posteam = ['season', 'week', 'game_id', 'posteam'] #posteam is the last item in the list

In [17]:
def group_cols(df, cols, agg_dict):
    """
    Function that groups columns with an aggregation dictionary
    """
    group_df = df.groupby(cols).agg(agg_dict).reset_index()
    return group_df

In [18]:
def group_cols_count(df, cols, output_col_name='count'):
    """
    Function that groups columns and counts the occurrences, outputting one column with a custom name.
    """
    group_df = df.groupby(cols).size().reset_index(name=output_col_name)
    return group_df

In [19]:
def expand_base_df(left, right, list):
    """
    This function merges our existing dataframe with a new one that essentially adds a column to it. It then fills the null values with 0.
    """
    base_df = pd.merge(left, right, on = list, how = 'left')
    base_df = base_df.fillna(0)
    return base_df

In [20]:
ints_sacks_agg = {'interception': 'sum', 'sack': 'sum', 'safety': 'sum'}

# Creating base dataframe
We'll start by grouping the defense dataframe so that there's an observation for every team in every game. The defteam variable gives us the plays in which that team was on defense. We'll group here for interception, sack and safety totals in each game for every team. Then we'll join the subsequent dataframes we create to this one.

In [21]:
base_df = group_cols(defense_df, cols_to_group_defteam, ints_sacks_agg)

In [22]:
base_df.head()

,season,week,game_id,defteam,interception,sack,safety
0,2022,1,2022_01_BAL_NYJ,BAL,1.0,3.0,0.0
1,2022,1,2022_01_BAL_NYJ,NYJ,1.0,2.0,0.0
2,2022,1,2022_01_BUF_LA,BUF,3.0,7.0,0.0
3,2022,1,2022_01_BUF_LA,LA,2.0,2.0,0.0
4,2022,1,2022_01_CLE_CAR,CAR,0.0,1.0,0.0


In [23]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266 entries, 0 to 1265
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   season        1266 non-null   int64  
 1   week          1266 non-null   int64  
 2   game_id       1266 non-null   object 
 3   defteam       1266 non-null   object 
 4   interception  1266 non-null   float64
 5   sack          1266 non-null   float64
 6   safety        1266 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 69.4+ KB


# Return touchdowns
There are lots of ways return touchdowns can be scored: interceptions, fumble recoveries, kickoffs, punts, blocked punts or field goals. We were going to filter for all of them individually, but since all of them are worth six points for a fantasy defense, we'll just filter for all return touchdowns. There are some cases where a kickoff was fumbled, recovered by the kicking team and returned for a touchdown. Something like this could be counted twice if we filtered for kickoff returns and fumble returns individually.

In [24]:
return_tds = defense_df[(defense_df['return_touchdown'] == 1)]

In [25]:
return_tds = group_cols_count(return_tds, cols_to_group_tdteam, 'return_tds')

In [26]:
return_tds.head()

,season,week,game_id,td_team,return_tds
0,2022,1,2022_01_NE_MIA,MIA,1
1,2022,1,2022_01_PHI_DET,PHI,1
2,2022,1,2022_01_PIT_CIN,PIT,1
3,2022,2,2022_02_ARI_LV,ARI,1
4,2022,2,2022_02_LAC_KC,KC,1


In [27]:
return_tds['return_tds'].value_counts()

return_tds
1    138
2     10
3      1
Name: count, dtype: int64

In [28]:
return_tds.sort_values(by = 'return_tds', ascending = False)

,season,week,game_id,td_team,return_tds
37,2022,13,2022_13_CLE_HOU,CLE,3
148,2024,4,2024_04_NO_ATL,ATL,2
85,2023,4,2023_04_NE_DAL,DAL,2
124,2023,16,2023_16_LV_KC,LV,2
61,2022,18,2022_18_NE_BUF,BUF,2
...,...,...,...,...,...
50,2022,16,2022_16_DEN_LA,LA,1
51,2022,16,2022_16_PHI_DAL,PHI,1
52,2022,17,2022_17_IND_NYG,NYG,1
53,2022,17,2022_17_JAX_HOU,JAX,1


Once in a while, we'll need to change the column name from td_team or posteam to defteam, which is the column name we'll use in our final dataframe. 

In [29]:
return_tds.rename(columns = {'td_team': 'defteam'}, inplace = True)

In [30]:
base_df = expand_base_df(base_df, return_tds, cols_to_group_defteam)

In [31]:
base_df.head()

,season,week,game_id,defteam,interception,sack,safety,return_tds
0,2022,1,2022_01_BAL_NYJ,BAL,1.0,3.0,0.0,0.0
1,2022,1,2022_01_BAL_NYJ,NYJ,1.0,2.0,0.0,0.0
2,2022,1,2022_01_BUF_LA,BUF,3.0,7.0,0.0,0.0
3,2022,1,2022_01_BUF_LA,LA,2.0,2.0,0.0,0.0
4,2022,1,2022_01_CLE_CAR,CAR,0.0,1.0,0.0,0.0


# Fumble recoveries

First we have to filter for defensive fumble recoveries on non-punts, because it seems that when a team punts, it's still considered the offensive team. So if the team receiving the punt loses a fumble, it's not credited to the defense. Teams kicking off are considered the defensive team. On non-punt plays, if defteam matches fumble_recovery_1_team or fumble_recovery_2_team, it should count as a fumble recovery for the defense.

In [32]:
fumble_df = defense_df[defense_df['fumble_lost'] == True]

In [33]:
fumble_no_punt = defense_df[(defense_df['fumble_lost'] == True) & (defense_df['punt_attempt'] == False)]

In [34]:
fumble_no_punt = fumble_no_punt[(fumble_no_punt['defteam'] == fumble_no_punt['fumble_recovery_1_team']) | (fumble_no_punt['defteam'] == fumble_no_punt['fumble_recovery_2_team'])]

In [35]:
fumble_recoveries = group_cols_count(fumble_no_punt, cols_to_group_defteam, 'fumble_recoveries')

In [36]:
fumble_recoveries.head()

,season,week,game_id,defteam,fumble_recoveries
0,2022,1,2022_01_BAL_NYJ,BAL,1
1,2022,1,2022_01_BUF_LA,LA,2
2,2022,1,2022_01_DEN_SEA,DEN,1
3,2022,1,2022_01_DEN_SEA,SEA,2
4,2022,1,2022_01_GB_MIN,MIN,1


In [37]:
base_df = expand_base_df(base_df, fumble_recoveries, cols_to_group_defteam)

Apparently, DraftKings awards a fumble recovery to a defense if the offensive team recovers a defensive fumble after fumbling a ball away to the defense. So we'll create a column for that.

In [38]:
fumble_rec_dk = fumble_no_punt[fumble_no_punt['posteam'] == fumble_no_punt['fumble_recovery_2_team']]

In [39]:
fumble_rec_dk = group_cols_count(fumble_rec_dk, cols_to_group_posteam, 'off_fum_rec_dk')

In [40]:
fumble_rec_dk.rename(columns = {'posteam': 'defteam'}, inplace = True)

In [41]:
base_df = expand_base_df(base_df, fumble_rec_dk, cols_to_group_defteam)

# Fumble recoveries on punts
Since a punting team is still considered the team with possession of the ball (posteam), if the team receiving the punt loses the ball on a fumble, we need to ensure that the fantasy defense is credited with a fumble recovery.

In [42]:
fumble_punt = defense_df[(defense_df['fumble_lost'] == True) & (defense_df['punt_attempt'] == True)]

In [43]:
fumble_punt = fumble_punt[(fumble_punt['posteam'] == fumble_punt['fumble_recovery_1_team'])\
| (fumble_punt['posteam'] == fumble_punt['fumble_recovery_2_team'])]

In [44]:
fumble_punt = group_cols_count(fumble_punt, cols_to_group_posteam, 'punt_fum_rec')

In [45]:
fumble_punt.rename(columns = {'posteam': 'defteam'}, inplace = True)

In [46]:
base_df = expand_base_df(base_df, fumble_punt, cols_to_group_defteam)

In [47]:
base_df.head()

,season,week,game_id,defteam,interception,sack,safety,return_tds,fumble_recoveries,off_fum_rec_dk,punt_fum_rec
0,2022,1,2022_01_BAL_NYJ,BAL,1.0,3.0,0.0,0.0,1.0,0.0,0.0
1,2022,1,2022_01_BAL_NYJ,NYJ,1.0,2.0,0.0,0.0,0.0,0.0,0.0
2,2022,1,2022_01_BUF_LA,BUF,3.0,7.0,0.0,0.0,0.0,0.0,0.0
3,2022,1,2022_01_BUF_LA,LA,2.0,2.0,0.0,0.0,2.0,0.0,0.0
4,2022,1,2022_01_CLE_CAR,CAR,0.0,1.0,0.0,0.0,0.0,0.0,0.0


# Blocked punts, field goals and extra points

In [48]:
blocked_punts = defense_df[defense_df['punt_blocked'] == True]

In [49]:
blocked_punts = group_cols_count(blocked_punts, cols_to_group_defteam, 'blocked_punts')

In [50]:
blocked_punts.head()

,season,week,game_id,defteam,blocked_punts
0,2022,2,2022_02_ATL_LA,ATL,1
1,2022,5,2022_05_DAL_LA,DAL,1
2,2022,6,2022_06_NYJ_GB,NYJ,1
3,2022,9,2022_09_IND_NE,NE,1
4,2022,9,2022_09_MIA_CHI,MIA,1


In [51]:
base_df = expand_base_df(base_df, blocked_punts, cols_to_group_defteam)

In [52]:
base_df.head()

,season,week,game_id,defteam,interception,sack,safety,return_tds,fumble_recoveries,off_fum_rec_dk,punt_fum_rec,blocked_punts
0,2022,1,2022_01_BAL_NYJ,BAL,1.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2022,1,2022_01_BAL_NYJ,NYJ,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022,1,2022_01_BUF_LA,BUF,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022,1,2022_01_BUF_LA,LA,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0
4,2022,1,2022_01_CLE_CAR,CAR,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
blocked_FGs = defense_df[(defense_df['field_goal_attempt'] == True) & (defense_df['field_goal_result'] == 'blocked')]

In [54]:
blocked_FGs = group_cols_count(blocked_FGs, cols_to_group_defteam, 'blocked_FG')

In [55]:
base_df = expand_base_df(base_df, blocked_FGs, cols_to_group_defteam)

In [56]:
blocked_XPs = defense_df[(defense_df['extra_point_attempt'] == True) & (defense_df['extra_point_result'] == 'blocked')]

In [57]:
blocked_XPs = group_cols_count(blocked_XPs, cols_to_group_defteam, 'blocked_XP')

In [58]:
base_df = expand_base_df(base_df, blocked_XPs, cols_to_group_defteam)

In [59]:
base_df.head()

,season,week,game_id,defteam,interception,sack,safety,return_tds,fumble_recoveries,off_fum_rec_dk,punt_fum_rec,blocked_punts,blocked_FG,blocked_XP
0,2022,1,2022_01_BAL_NYJ,BAL,1.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2022,1,2022_01_BAL_NYJ,NYJ,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022,1,2022_01_BUF_LA,BUF,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022,1,2022_01_BUF_LA,LA,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,2022,1,2022_01_CLE_CAR,CAR,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Defensive 2-point conversion returns
We've only had 12 of these since 2000.

In [60]:
conv_returns = defense_df[(defense_df['defensive_two_point_conv'] == True)\
| (defense_df['defensive_extra_point_conv'] == True)].reset_index()

In [61]:
conv_returns = conv_returns[['season', 'week', 'game_id', 'defteam', 'defensive_two_point_conv', 'defensive_extra_point_conv']]

In [62]:
#No need to group here. These all happened in different games

In [63]:
base_df = expand_base_df(base_df, conv_returns, cols_to_group_defteam)

In [64]:
base_df.head()

,season,week,game_id,defteam,interception,sack,safety,return_tds,fumble_recoveries,off_fum_rec_dk,punt_fum_rec,blocked_punts,blocked_FG,blocked_XP,defensive_two_point_conv,defensive_extra_point_conv
0,2022,1,2022_01_BAL_NYJ,BAL,1.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022,1,2022_01_BAL_NYJ,NYJ,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022,1,2022_01_BUF_LA,BUF,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022,1,2022_01_BUF_LA,LA,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022,1,2022_01_CLE_CAR,CAR,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
#Next: Points Allowed!

# Final scores
We'll create a dataframe with the final scores for each home team and away team for every game, and we'll also indicate the home and away team.

In [66]:
final_scores = defense_df.groupby(['season', 'week', 'game_id']).agg({'away_score': 'max', 'home_score': 'max'}).reset_index()

In [67]:
final_scores.tail()

,season,week,game_id,away_score,home_score
628,2024,4,2024_04_PHI_TB,16,33
629,2024,4,2024_04_PIT_IND,24,27
630,2024,4,2024_04_SEA_DET,29,42
631,2024,4,2024_04_TEN_MIA,31,12
632,2024,4,2024_04_WAS_ARI,42,14


In [68]:
home_away_df = defense_df[['season', 'week', 'game_id', 'away_team', 'home_team']]

In [69]:
final_scores = pd.merge(final_scores, home_away_df, on = ['season', 'week', 'game_id'], how = 'left')

In [70]:
final_scores = final_scores.drop_duplicates()

In [71]:
final_scores.tail()

,season,week,game_id,away_score,home_score,away_team,home_team
109169,2024,4,2024_04_PHI_TB,16,33,PHI,TB
109342,2024,4,2024_04_PIT_IND,24,27,PIT,IND
109518,2024,4,2024_04_SEA_DET,29,42,SEA,DET
109703,2024,4,2024_04_TEN_MIA,31,12,TEN,MIA
109875,2024,4,2024_04_WAS_ARI,42,14,WAS,ARI


# Scoring change dataframe
Filtering each row of the play-by-play data for each scoring change in which the score changes for the team on defense, and then we'll filter the same thing for offense. Then we'll concatenate into one dataframe and sort by the index so all the plays are in sequence.

In [72]:
def_score_changes = defense_df[defense_df['defteam_score'] < defense_df['defteam_score_post']]

In [73]:
off_score_changes = defense_df[defense_df['posteam_score'] < defense_df['posteam_score_post']]

In [74]:
all_score_changes = pd.concat([def_score_changes, off_score_changes])

In [75]:
all_score_changes = all_score_changes.sort_index()

In [76]:
#all_score_changes = all_score_changes.sort_values(by = ['season', 'week', 'game_id', 'away_score', 'home_score'])

In [77]:
all_score_changes['has_ball'] = np.where(all_score_changes['away_team'] == all_score_changes['posteam'], 'away', 'home')

In [78]:
len(all_score_changes)

7998

# Functions to create columns indicating scores against a fantasy defense
We need to account for touchdowns, field goals, extra points and 2-point conversions.

In [79]:
def score_vs_def_col(diff, poss):
    """
    This function creates columns that indicate whether a touchdown or field goal was scored 
    against a fantasy defense on that particular play. The diff argument is the scoring difference, three or six points.
    The poss is the team that has the ball.
    """
    new_column = np.where(((all_score_changes['posteam_score_post'] - all_score_changes['posteam_score']) == diff)\
                          & (all_score_changes['has_ball'] == poss), 1, 0)
    return new_column

In [80]:
def punt_td_vs_def(poss):
    """
    This function creates columns to indicate whether a punt was returned for a touchdown. Even though the defensive unit isn't on the field,
    this still should count as points against a fantasy defense. We need to write this function separately because the data indicates
    the defensive team as the scoring team when punts are returned for touchdowns.
    """
    new_column = np.where(((all_score_changes['defteam_score_post'] - all_score_changes['defteam_score']) == 6)\
                          & (all_score_changes['punt_attempt'] == True) & (all_score_changes['has_ball'] != poss), 1, 0)
    return new_column

In [81]:
all_score_changes['td_vs_H'] = score_vs_def_col(6, 'away')

In [82]:
all_score_changes['fg_vs_H'] = score_vs_def_col(3, 'away')

In [83]:
all_score_changes['td_vs_A'] = score_vs_def_col(6, 'home')

In [84]:
all_score_changes['fg_vs_A'] = score_vs_def_col(3, 'home')

In [85]:
all_score_changes['punt_TD_vs_H'] = punt_td_vs_def('away')

In [86]:
all_score_changes['punt_TD_vs_A'] = punt_td_vs_def('home')

In [87]:
def xp_vs_def_col(poss):
    """
    This function rounds up all the successful extra points that count against a fantasy defense.
    It checks that the possession team's score has increased by one and also checks that the possession team's score
    increased by six in the previous row. Otherwise, it was a defensive touchdown and the XP wouldn't be charged to the fantasy defense.
    """
    new_column = np.where(((all_score_changes['posteam_score_post'] - all_score_changes['posteam_score']) == 1)\
    & ((all_score_changes['posteam_score_post'].shift(1) - all_score_changes['posteam_score'].shift(1)) == 6)\
                          & ((all_score_changes['has_ball'] == poss)), 1, 0)  
    return new_column

In [88]:
def xp2_vs_def_col(poss):
    """
    This function rounds up all the successful 2-point conversions that count against a fantasy defense.
    It checks that the possession team's score has increased by two and also checks that the possession team's score
    increased by six in the previous row. Otherwise, it was a defensive touchdown and the XP wouldn't be charged to the fantasy defense.
    """
    new_column = np.where(((all_score_changes['posteam_score_post'] - all_score_changes['posteam_score']) == 2)\
    & ((all_score_changes['posteam_score_post'].shift(1) - all_score_changes['posteam_score'].shift(1)) == 6)\
                          & (all_score_changes['has_ball'] == poss), 1, 0)  
    return new_column

In [89]:
def xp_after_punt(poss):
    """
    This function creates a column to indicate successful extra points after a punt is returned for a touchdown. This time,
    it checks that the defensive team's score increases by six in the previous row and also that the play was a punt attempt.
    """
    new_column = np.where(((all_score_changes['posteam_score_post'] - all_score_changes['posteam_score']) == 1)\
                          & ((all_score_changes['defteam_score_post'].shift(1) - all_score_changes['defteam_score'].shift(1)) == 6)\
                          & (all_score_changes['punt_attempt'].shift(1) == True)\
                          & (all_score_changes['has_ball'] == poss), 1, 0)  
    return new_column

In [90]:
def xp2_after_punt(poss):
    """
    This function creates a column to indicate successful 2-point conversions after a punt is returned for a touchdown. This time,
    it checks that the defensive team's score increases by six in the previous row and also that the play was a punt attempt.
    """
    new_column = np.where(((all_score_changes['posteam_score_post'] - all_score_changes['posteam_score']) == 2)\
                          & ((all_score_changes['defteam_score_post'].shift(1) - all_score_changes['defteam_score'].shift(1)) == 6)\
                          & (all_score_changes['punt_attempt'].shift(1) == True)\
                          & (all_score_changes['has_ball'] == poss), 1, 0)  
    return new_column

In [91]:
all_score_changes['xp_vs_H'] = xp_vs_def_col('away')

In [92]:
all_score_changes['xp_vs_A'] = xp_vs_def_col('home')

In [93]:
all_score_changes['xp2_vs_H'] = xp2_vs_def_col('away')

In [94]:
all_score_changes['xp2_vs_A'] = xp2_vs_def_col('home')

In [95]:
all_score_changes['xp_after_punt_vs_H'] = xp_after_punt('away')

In [96]:
all_score_changes['xp_after_punt_vs_A'] = xp_after_punt('home')

In [97]:
all_score_changes['xp2_after_punt_vs_H'] = xp2_after_punt('away')

In [98]:
all_score_changes['xp2_after_punt_vs_A'] = xp2_after_punt('home')

In [99]:
len(all_score_changes)

7998

In [100]:
all_score_changes = all_score_changes[['season', 'week', 'game_id', 'away_team', 'home_team', 'td_vs_H', 'td_vs_A',\
                                      'fg_vs_H', 'fg_vs_A', 'punt_TD_vs_H', 'punt_TD_vs_A', 'xp_vs_H', 'xp_vs_A',\
                                      'xp2_vs_H', 'xp2_vs_A', 'xp_after_punt_vs_H', 'xp_after_punt_vs_A',\
                                      'xp2_after_punt_vs_H', 'xp2_after_punt_vs_A']]

In [101]:
all_score_changes = all_score_changes.groupby(['season', 'week', 'game_id', 'away_team', 'home_team']).sum().reset_index()

In [102]:
len(all_score_changes)

633

# Combine columns function
We'll need to consolidate some of the columns, so that touchdowns on non-punts and touchdowns on punts are in the same column.

In [103]:
def combine_cols(df, new_col_name, *args):
    """
    We can consolidate a lot of the columns in the all_score_changes dataframes as well as the base_df.
    For example, td_vs_A and punt_td_vs_A can be added together into one column.
    This function will sum the column values and then drop the columns we don't need
    """
    df[new_col_name] = df[list(args)].sum(axis = 1)
    df.drop(columns = list(args), inplace = True)
    return df

In [104]:
all_score_changes.tail()

,season,week,game_id,away_team,home_team,td_vs_H,td_vs_A,fg_vs_H,fg_vs_A,punt_TD_vs_H,punt_TD_vs_A,xp_vs_H,xp_vs_A,xp2_vs_H,xp2_vs_A,xp_after_punt_vs_H,xp_after_punt_vs_A,xp2_after_punt_vs_H,xp2_after_punt_vs_A
628,2024,4,2024_04_PHI_TB,PHI,TB,2,4,0,2,0,0,2,3,0,0,0,0,0,0
629,2024,4,2024_04_PIT_IND,PIT,IND,3,3,1,2,0,0,3,3,0,0,0,0,0,0
630,2024,4,2024_04_SEA_DET,SEA,DET,4,6,0,0,0,0,3,6,0,0,0,0,0,0
631,2024,4,2024_04_TEN_MIA,TEN,MIA,2,1,5,2,0,0,2,0,0,0,0,0,0,0
632,2024,4,2024_04_WAS_ARI,WAS,ARI,5,2,2,0,0,0,4,2,1,0,0,0,0,0


In [105]:
all_score_changes = combine_cols(all_score_changes, 'TD_vs_H', 'td_vs_H', 'punt_TD_vs_H')

In [106]:
all_score_changes = combine_cols(all_score_changes, 'TD_vs_A', 'td_vs_A', 'punt_TD_vs_A')

In [107]:
all_score_changes = combine_cols(all_score_changes, 'XP_vs_H', 'xp_vs_H', 'xp_after_punt_vs_H')

In [108]:
all_score_changes = combine_cols(all_score_changes, 'XP_vs_A', 'xp_vs_A', 'xp_after_punt_vs_A')

In [109]:
all_score_changes = combine_cols(all_score_changes, 'XP2_vs_H', 'xp2_vs_H', 'xp2_after_punt_vs_H')

In [110]:
all_score_changes = combine_cols(all_score_changes, 'XP2_vs_A', 'xp2_vs_A', 'xp2_after_punt_vs_A')

In [111]:
all_score_changes.tail()

,season,week,game_id,away_team,home_team,fg_vs_H,fg_vs_A,TD_vs_H,TD_vs_A,XP_vs_H,XP_vs_A,XP2_vs_H,XP2_vs_A
628,2024,4,2024_04_PHI_TB,PHI,TB,0,2,2,4,2,3,0,0
629,2024,4,2024_04_PIT_IND,PIT,IND,1,2,3,3,3,3,0,0
630,2024,4,2024_04_SEA_DET,SEA,DET,0,0,4,6,3,6,0,0
631,2024,4,2024_04_TEN_MIA,TEN,MIA,5,2,2,1,2,0,0,0
632,2024,4,2024_04_WAS_ARI,WAS,ARI,2,0,5,2,4,2,1,0


# Getting one row for each team in each game
We currently have one row for each game, with points against home teams and away teams in separate columns. Now we'll turn this into a dataframe with two rows for each game, one for the home team and one for the away team.<br>

Then we'll multiply the values by the appropriate points, and use np.select to convert the raw point total into the corresponding fantasy points.

In [112]:
all_score_changes_away = all_score_changes.copy()
all_score_changes_home = all_score_changes.copy()

In [113]:
# Assign new column 'def_team' and keep only necessary columns for each new row
all_score_changes_away['defteam'] = all_score_changes_away['away_team']
all_score_changes_away = all_score_changes_away[['season', 'week', 'game_id', 'defteam', 'fg_vs_A', 'TD_vs_A', 'XP_vs_A', 'XP2_vs_A']]
all_score_changes_away.columns = ['season', 'week', 'game_id', 'defteam', 'FG_allowed', 'TD_allowed', 'XP_allowed', 'XP2_allowed']

all_score_changes_home['defteam'] = all_score_changes_home['home_team']
all_score_changes_home = all_score_changes_home[['season', 'week', 'game_id', 'defteam', 'fg_vs_H', 'TD_vs_H', 'XP_vs_H', 'XP2_vs_H']]
all_score_changes_home.columns = ['season', 'week', 'game_id', 'defteam', 'FG_allowed', 'TD_allowed', 'XP_allowed', 'XP2_allowed']

In [114]:
# Concatenate the rows back together
scoring_vs_def = pd.concat([all_score_changes_away, all_score_changes_home])

In [115]:
# Sort by season, week, and game_id to maintain the original order
scoring_vs_def = scoring_vs_def.sort_values(by=['season', 'week', 'game_id']).reset_index(drop=True)

In [116]:
scoring_vs_def['TD_allowed'] = scoring_vs_def['TD_allowed'] * 6
scoring_vs_def['FG_allowed'] = scoring_vs_def['FG_allowed'] * 3
scoring_vs_def['XP2_allowed'] = scoring_vs_def['XP2_allowed'] * 2

In [117]:
scoring_vs_def['points_against'] = scoring_vs_def['TD_allowed'] + scoring_vs_def['FG_allowed'] + scoring_vs_def['XP_allowed'] + scoring_vs_def['XP2_allowed']

In [118]:
scoring_vs_def.drop(columns = ['TD_allowed', 'FG_allowed', 'XP2_allowed', 'XP_allowed'], inplace = True)

In [119]:
conditions = [
    (scoring_vs_def['points_against'] == 0),
    (scoring_vs_def['points_against'] >= 1) & (scoring_vs_def['points_against'] <= 6),
    (scoring_vs_def['points_against'] >= 7) & (scoring_vs_def['points_against'] <= 13),
    (scoring_vs_def['points_against'] >= 14) & (scoring_vs_def['points_against'] <= 20),
    (scoring_vs_def['points_against'] >= 21) & (scoring_vs_def['points_against'] <= 27),
    (scoring_vs_def['points_against'] >= 28) & (scoring_vs_def['points_against'] <= 34),
    (scoring_vs_def['points_against'] >= 35)
]

# Define corresponding values
values = [10, 7, 4, 1, 0, -1, -4]

# Apply np.select to create the new column with updated values
scoring_vs_def['points_against'] = np.select(conditions, values)

In [120]:
scoring_vs_def.head()

,season,week,game_id,defteam,points_against
0,2022,1,2022_01_BAL_NYJ,BAL,4
1,2022,1,2022_01_BAL_NYJ,NYJ,0
2,2022,1,2022_01_BUF_LA,BUF,4
3,2022,1,2022_01_BUF_LA,LA,-1
4,2022,1,2022_01_CLE_CAR,CLE,0


In [121]:
base_df.head()

,season,week,game_id,defteam,interception,sack,safety,return_tds,fumble_recoveries,off_fum_rec_dk,punt_fum_rec,blocked_punts,blocked_FG,blocked_XP,defensive_two_point_conv,defensive_extra_point_conv
0,2022,1,2022_01_BAL_NYJ,BAL,1.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022,1,2022_01_BAL_NYJ,NYJ,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022,1,2022_01_BUF_LA,BUF,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022,1,2022_01_BUF_LA,LA,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022,1,2022_01_CLE_CAR,CAR,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
base_df = combine_cols(base_df, 'fumble_rec', 'fumble_recoveries', 'punt_fum_rec')

In [123]:
base_df = combine_cols(base_df, 'blocked_kicks', 'blocked_punts', 'blocked_FG', 'blocked_XP')

In [124]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266 entries, 0 to 1265
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   season                      1266 non-null   int64  
 1   week                        1266 non-null   int64  
 2   game_id                     1266 non-null   object 
 3   defteam                     1266 non-null   object 
 4   interception                1266 non-null   float64
 5   sack                        1266 non-null   float64
 6   safety                      1266 non-null   float64
 7   return_tds                  1266 non-null   float64
 8   off_fum_rec_dk              1266 non-null   float64
 9   defensive_two_point_conv    1266 non-null   float64
 10  defensive_extra_point_conv  1266 non-null   float64
 11  fumble_rec                  1266 non-null   float64
 12  blocked_kicks               1266 non-null   float64
dtypes: float64(9), int64(2), object(2

Here the scoring_vs_def column is merged with the base_df. Then we'll apply the scoring system to get actual fantasy points scored for every team's defense in every game.

In [125]:
base_df = expand_base_df(base_df, scoring_vs_def, cols_to_group_defteam)

In [126]:
base_df.head()

,season,week,game_id,defteam,interception,sack,safety,return_tds,off_fum_rec_dk,defensive_two_point_conv,defensive_extra_point_conv,fumble_rec,blocked_kicks,points_against
0,2022,1,2022_01_BAL_NYJ,BAL,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
1,2022,1,2022_01_BAL_NYJ,NYJ,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2022,1,2022_01_BUF_LA,BUF,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,2022,1,2022_01_BUF_LA,LA,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,-1
4,2022,1,2022_01_CLE_CAR,CAR,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [127]:
base_df['FD_Pts'] = base_df['sack'] + (base_df['fumble_rec'] * 2) + (base_df['return_tds'] * 6)\
+ (base_df['defensive_two_point_conv'] * 2) + (base_df['safety'] * 2) + (base_df['blocked_kicks'] * 2)\
+ (base_df['interception'] * 2) + base_df['points_against']

In [128]:
base_df['DK_Pts'] = base_df['sack'] + (base_df['fumble_rec'] * 2) + (base_df['return_tds'] * 6)\
+ (base_df['defensive_two_point_conv'] * 2) + (base_df['safety'] * 2) + (base_df['blocked_kicks'] * 2)\
+ (base_df['interception'] * 2) + base_df['off_fum_rec_dk'] + base_df['points_against']

In [129]:
def_target_var = base_df[['season', 'week', 'game_id', 'defteam', 'FD_Pts', 'DK_Pts']]

In [130]:
def_target_var.tail()

,season,week,game_id,defteam,FD_Pts,DK_Pts
1261,2024,4,2024_04_SEA_DET,SEA,1.0,1.0
1262,2024,4,2024_04_TEN_MIA,MIA,2.0,2.0
1263,2024,4,2024_04_TEN_MIA,TEN,10.0,10.0
1264,2024,4,2024_04_WAS_ARI,ARI,-2.0,-2.0
1265,2024,4,2024_04_WAS_ARI,WAS,7.0,7.0


In [131]:
def_target_var.to_csv('def_points_update_' + str(season) + '_' + str(week) + '.csv', index = False)

Adding the target variable dataframe to the database

In [132]:
# conn = sqlite3.connect('nfl_dfs.db')

# # Define the table name
# table_name = 'defense_target'

# # Specify data types
# dtype = {
#     'season': 'INTEGER',
#     'week': 'INTEGER',
#     'game_id': 'TEXT',
#     'defteam': 'TEXT',
#     'FD_Pts': 'REAL',
#     'DK_Pts': 'REAL'
# }

# # Write the dataframe to the SQLite table
# def_target_var.to_sql(table_name, conn, if_exists='replace', index=False, dtype=dtype)

# # Confirm that the data has been written
# print(f"Data written to table {table_name} in SQLite database nfl_fantasy.db")

# # Close the connection
# conn.close()

In [133]:
# # Connect to SQLite database
# conn = sqlite3.connect('nfl_dfs.db')

# # Create a cursor object
# cursor = conn.cursor()

# # Execute a query to get the list of tables
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# # Fetch all results
# tables = cursor.fetchall()

# # Print the list of tables
# print("Tables in the database:")
# for table in tables:
#     print(table[0])

# # Close the connection
# conn.close()